<a href="https://colab.research.google.com/github/leslie-zi-pan/kaggle-courses/blob/main/ryanluoli2_notebooks/Get_Started_with_a_Simple_Random_Forest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get Started with a Simple Random Forest on Spaceship Titanic Dataset

https://www.kaggle.com/competitions/spaceship-titanic/overview


# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Kaggle data setup

### Authorise with kaggle json api key

In [ ]:
! pip install kaggle
! rm -rf ~/.kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/kaggle/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### Download kaggle dataset

In [ ]:
!kaggle competitions download -c spaceship-titanic
# !kaggle datasets download -d dansbecker/melbourne-housing-snapshot
!unzip spaceship-titanic

  0% 0.00/299k [00:00<?, ?B/s]
100% 299k/299k [00:00<00:00, 88.2MB/s]
Archive:  spaceship-titanic.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


## Imports

In [ ]:
import os
import numpy as np
import pandas as pd

import seaborn as sn 
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate

print('Setup Complete')

Setup Complete


## Load the data

In [ ]:
# Path of the file to read
train_path = '/content/train.csv'
test_path = '/content/test.csv'
submission_path = '/content/sample_submission.csv'

# Read the file into a variable
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)
submission = pd.read_csv(submission_path)

## Examine data

In [ ]:
df_train.head(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
5,0005_01,Earth,False,F/0/P,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,Sandie Hinetthews,True
6,0006_01,Earth,False,F/2/S,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,Billex Jacostaffey,True
7,0006_02,Earth,True,G/0/S,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,Candra Jacostaffey,True
8,0007_01,Earth,False,F/3/S,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,Andona Beston,True
9,0008_01,Europa,True,B/1/P,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,Erraiam Flatic,True


In [ ]:
# Print last rows using tail() function
df_train.tail(10)

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
8683,9272_02,Earth,False,F/1894/P,TRAPPIST-1e,21.0,False,86.0,3.0,149.0,208.0,329.0,Gordo Simson,False
8684,9274_01,NaN,True,G/1508/P,TRAPPIST-1e,23.0,False,0.0,0.0,0.0,0.0,0.0,Chelsa Bullisey,True
8685,9275_01,Europa,False,A/97/P,TRAPPIST-1e,0.0,False,0.0,0.0,0.0,0.0,0.0,Polaton Conable,True
8686,9275_02,Europa,False,A/97/P,TRAPPIST-1e,32.0,False,1.0,1146.0,0.0,50.0,34.0,Diram Conable,False
8687,9275_03,Europa,NaN,A/97/P,TRAPPIST-1e,30.0,False,0.0,3208.0,0.0,2.0,330.0,Atlasym Conable,True
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False
8692,9280_02,Europa,False,E/608/S,TRAPPIST-1e,44.0,False,126.0,4688.0,0.0,0.0,12.0,Propsh Hontichre,True


In [ ]:
df_train.describe()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
count,8514.000000,8512.000000,8510.000000,8485.000000,8510.000000,8505.000000
mean,28.827930,224.687617,458.077203,173.729169,311.138778,304.854791
std,14.489021,666.717663,1611.489240,604.696458,1136.705535,1145.717189
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,27.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,38.000000,47.000000,76.000000,27.000000,59.000000,46.000000
max,79.000000,14327.000000,29813.000000,23492.000000,22408.000000,24133.000000


In [ ]:
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

## Data Preparation


In [ ]:
cat_features = df_train.columns[df_train.dtypes == 'object']
num_features = df_train.columns[df_train.dtypes != 'object']

In [ ]:
df_train['Transported'] = df_train['Transported'].map({True: 1, False: 0})

In [ ]:
df = pd.concat([df_train, df_test], copy=0).copy()

In [ ]:
df1 = df.copy()
df1 = df1.drop(['Cabin', 'Name'], axis=1)

In [ ]:
df1.sort_values(by='PassengerId').head(10)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,0.0
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,1.0
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0.0
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0.0
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,1.0
5,0005_01,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,1.0
6,0006_01,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,1.0
7,0006_02,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,1.0
8,0007_01,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,1.0
9,0008_01,Europa,True,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,1.0


### Feature Engineering


In [ ]:
df2 = df1.copy()

df2['group'] = df2.apply(lambda row: row['PassengerId'][0:4], axis=1)
df2['number'] = df2.apply(lambda row: int(row['PassengerId'][5:7]), axis=1)
df2['FamSize'] = df2.groupby(['group'])['number'].transform(np.sum)

# df2 = df2.drop(['PassengerId','group','number'], axis=1)

features = df2.columns.tolist()
features.remove('Transported')
features.append('Transported')
df2 = df2[features].copy()
df2.sort_values(by='group').head(10)

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,group,number,FamSize,Transported
0,0001_01,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,0001,1,1,0.0
1,0002_01,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,0002,1,1,1.0
2,0003_01,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,0003,1,3,0.0
3,0003_02,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,0003,2,3,0.0
4,0004_01,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,0004,1,1,1.0
5,0005_01,Earth,False,PSO J318.5-22,44.0,False,0.0,483.0,0.0,291.0,0.0,0005,1,1,1.0
6,0006_01,Earth,False,TRAPPIST-1e,26.0,False,42.0,1539.0,3.0,0.0,0.0,0006,1,3,1.0
7,0006_02,Earth,True,TRAPPIST-1e,28.0,False,0.0,0.0,0.0,0.0,NaN,0006,2,3,1.0
8,0007_01,Earth,False,TRAPPIST-1e,35.0,False,0.0,785.0,17.0,216.0,0.0,0007,1,1,1.0
9,0008_01,Europa,True,55 Cancri e,14.0,False,0.0,0.0,0.0,0.0,0.0,0008,1,6,1.0


In [ ]:
df2 = df1.copy()

# Sort out family sizing and grouping
df2['group'] = df2.apply(lambda row: row['PassengerId'].split('_')[0], axis=1)
df2['FamSize'] = df2.groupby(['group'])['group'].transform('count')

df2 = df2.drop(['PassengerId','group'], axis=1)

features = df2.columns.tolist()
features.remove('Transported')
features.append('Transported')
df2 = df2[features].copy()

### Standardize Data

In [ ]:
num_missing_features = df2.columns[5:-1].to_list()
num_missing_features.append('Age')

cat_missing_features = df2.columns[:5].to_list()
cat_missing_features.remove('Age')

In [ ]:
df3 = df2.copy()

scaler = StandardScaler()
df3[num_missing_features] = scaler.fit_transform(df3[num_missing_features])

### Deal with Missing Values

In [ ]:
df3.isnull().sum()

HomePlanet       288
CryoSleep        310
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
FamSize            0
Transported     4277
dtype: int64

In [ ]:
df4 = df3.copy()

imputer = KNNImputer(n_neighbors=3)
df4[num_missing_features] = imputer.fit_transform(df4[num_missing_features])

In [ ]:
df5 = df4.copy()

for feature in cat_missing_features:
    df5[feature] = df5[feature].fillna(value=df5[feature].mode()[0])

In [ ]:
df5.isnull().sum()

HomePlanet         0
CryoSleep          0
Destination        0
Age                0
VIP                0
RoomService        0
FoodCourt          0
ShoppingMall       0
Spa                0
VRDeck             0
FamSize            0
Transported     4277
dtype: int64

### Encoding Categorical Variables

In [ ]:
df6 = df5.copy()

for feature in ['HomePlanet','Destination']:
    df6 = pd.get_dummies(df6, columns=[feature])
    df6 = df6.drop([df6.columns[-1]], axis=1)

for feature in ['CryoSleep','VIP']:
    df6[feature] = df6[feature].map({True:1, False:0})

### Split Data

In [ ]:
X_train = df6.iloc[:df_train.shape[0],:].copy()
X_train = X_train.drop(['Transported'], axis=1).copy()
y_train = df_train['Transported'].copy()

X_test = df6.iloc[df_train.shape[0]:,:]

## Random Forest

In [ ]:
parameters = {
    'n_estimators': [200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 4, 6],
    'min_samples_leaf': [2, 4, 6],
    'random_state':[0]
              }

In [ ]:
rf_model = RandomForestClassifier()
clf = GridSearchCV(rf_model, parameters)
clf.fit(X_train, y_train)

GridSearchCV(estimator=RandomForestClassifier(),
             param_grid={'max_depth': [10, 20], 'min_samples_leaf': [2, 4, 6],
                         'min_samples_split': [2, 4, 6], 'n_estimators': [200],
                         'random_state': [0]})

In [ ]:
clf.best_params_

{'max_depth': 20,
 'min_samples_leaf': 2,
 'min_samples_split': 6,
 'n_estimators': 200,
 'random_state': 0}

In [ ]:
best_rf_model = clf.best_estimator_
rf_cv_scores = cross_validate(best_rf_model, X_train, y_train, scoring='accuracy', cv=5)
round(rf_cv_scores['test_score'].mean(),5)

0.79869

## Make Prediction


In [ ]:
y_pred = best_rf_model.predict(X_test.drop(['Transported'],axis=1))
submission['Transported'] = y_pred
submission['Transported'] = submission['Transported'].map({1:True, 0:False})
submission.head()

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [ ]:
submission.to_csv('submission.csv', index=False)